# Combining order_products and products

## Contains:
### Merging orders_products and products clean

In [1]:
#import libraries
import pandas as pd
import numpy as np
import os

In [2]:
#create text file path
path = r'C:\Users\krist\08.2023_InstacartBasketAnalysis'

## import both datasets

In [8]:
#import orders_products_combined and cleaned products
#after visualizing added index_col = [0] to get rid of redundant column
df_orders_products_combined = pd.read_pickle(os.path.join(path, 
                                                          '02.Data', 
                                                          'Prepared_Data', 
                                                          'orders_products_combined.pkl'))
df_products_clean = pd.read_csv(os.path.join(path, 
                                             '02.Data', 
                                             'Prepared_Data', 
                                             'products_checked.csv'), 
                                                index_col = [0])

In [5]:
#check output of orders_products_combined
df_orders_products_combined.head()

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,_merge
0,2539329,1,1,2,8,11.0,196,1,0,both
1,2539329,1,1,2,8,11.0,14084,2,0,both
2,2539329,1,1,2,8,11.0,12427,3,0,both
3,2539329,1,1,2,8,11.0,26088,4,0,both
4,2539329,1,1,2,8,11.0,26405,5,0,both


In [6]:
#check shape of orders_products_combined
df_orders_products_combined.shape

(32434489, 10)

### shape of orders_products_combined is same as what was exported 32,434,489 rows and 10 columns

In [9]:
#check output of products_clean
df_products_clean.head()

,product_id,product_name,aisle_id,department_id,prices
0,1,Chocolate Sandwich Cookies,61,19,5.8
1,2,All-Seasons Salt,104,13,9.3
2,3,Robust Golden Unsweetened Oolong Tea,94,7,4.5
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1,10.5
4,5,Green Chile Anytime Sauce,5,13,4.3


In [10]:
#check shape of products_clean
df_products_clean.shape

(49672, 5)

### datasets share product_id column and no other. join on product_id

## Merge datasets

In [12]:
#combine datasets using inner join and indicator
#used inner join, assume only want data where there is shared product_id
df_merged_large = df_orders_products_combined.merge(df_products_clean, 
                                                    on = 'product_id', 
                                                    indicator = '_exists')

### got error when used indicator = True. used indicator = '_exists' to create new column for merge flag

In [13]:
#check output 
df_merged_large.head()

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,_merge,product_name,aisle_id,department_id,prices,_exists
0,2539329,1,1,2,8,11.0,196,1,0,both,Soda,77,7,9.0,both
1,2398795,1,2,3,7,15.0,196,1,1,both,Soda,77,7,9.0,both
2,473747,1,3,3,12,21.0,196,1,1,both,Soda,77,7,9.0,both
3,2254736,1,4,4,7,29.0,196,1,1,both,Soda,77,7,9.0,both
4,431534,1,5,4,15,28.0,196,1,1,both,Soda,77,7,9.0,both


In [14]:
#check merge flag, should be all both, zero left_only, zero right_only d/t inner join
df_merged_large['_exists'].value_counts()

both          32404859
left_only            0
right_only           0
Name: _exists, dtype: int64

In [15]:
#combine datasets
#used outer join, to check merge flag
df_merged_large_outer = df_orders_products_combined.merge(df_products_clean, 
                                                          on = 'product_id', 
                                                          indicator = '_exists', 
                                                          how = 'outer')

In [16]:
#double check merge to see if there is full match
df_merged_large_outer['_exists'].value_counts()

both          32404859
left_only        30200
right_only          11
Name: _exists, dtype: int64

### not a full match because 30,200 left_only and 11 right_only. 32,404,859 were both and this is how many are both from the inner join

## Check on shape of new combined dataset

In [17]:
df_merged_large.shape

(32404859, 15)

### new combined dataset has 32,404,859 rows and 15 columns

In [18]:
#export combined dataset to pkl file for faster import and export
df_merged_large.to_pickle(os.path.join(path, 
                                       '02.Data', 
                                       'Prepared_Data', 
                                       'orders_products_merged.pkl'))